In [ ]:
%matplotlib inline

import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt

from pyReef.model import Model 

In [ ]:
# initialise model
reef = Model()

In [ ]:
# Define the XmL input file
reef.load_xml('input.xml')

In [ ]:
print reef.pyGrid.partIDs

In [ ]:
print reef.pyGrid.regZ.shape,reef.pyGrid.partIDs
print reef.force.waveNb,reef.force.waveWu

In [ ]:
# display plots in SVG format
%config InlineBackend.figure_format = 'svg' 

plt.figure(figsize = (5,5))
ax = plt.gca()

# Map
im = ax.imshow(reef.pyGrid.regZ.T, extent=(0,1,0,1), origin='lower', aspect=1)
im.set_cmap('terrain')

# Colorbar
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="10%", pad=0.1, aspect=20)
cb = plt.colorbar(im, cax=cax)
cb.set_label('topography',size=12)

plt.show()

In [ ]:
reef.pyGrid.regZ.shape

In [ ]:
reef.pyGrid.regX.shape

In [ ]:
reef.pyGrid.regY.shape

In [ ]:
x,y = np.meshgrid(reef.pyGrid.regX, reef.pyGrid.regY)

In [ ]:
x.shape

In [3]:
!mpiexec -n 7 python mpi_example.py 10000